In [2]:
#This script prepares training and test data for trip duration prediction.
#It reads in and merges cleaned citi bike trip data from each month together, and add
#weather information to each date. 

import pandas as pd
import numpy as np
import psycopg2

#This function extracts date and time from timestamp object. 
def extract_date_time(input):
    dt = input.date()
    date = f"{dt:%Y-%m-%d}"
    time = input.hour
    return date,time

#This function retrieves hourly weather information from PostgreSQL database and extracts date and time bucket.
def weather_history():
    conn = psycopg2.connect(dbname='nycweather',user='postgres',password='Qzn651213')
    raw = pd.read_sql("SELECT time,precipintensity,temperature FROM weather_newyork;", conn)
    raw['Date'], raw['Start_Time'] = zip(*raw['time'].map(extract_date_time))
    raw = raw.drop(columns=['time'])
    return raw

#This function reads cleaned data with relevant columns.
def read_cleaned(filename):
    raw = pd.read_csv(filename,usecols=[1,2,4,5,8,9,10,11])
    return raw

def add_weather_save(database,save_filename):
    global history 
    raw = database
    
    #associate weather data to date and time
    cols = ['Start_Time', 'Date']
    raw = raw.join(history.set_index(cols), on=cols)
    
    #drop unnecessary columns and rename columns. 
    raw = raw.drop(columns=['Date'])
    raw = raw.rename(columns={'precipintensity': 'Precipitation', 'temperature': 'Temperature'})
    
    #save final data for training and evaluation
    raw.to_csv(save_filename,index=False)

    
if __name__ == '__main__':
    
    #retrieve weather history as pandas dataframe    
    history = weather_history()
    
    #database column name 
    database_name = ['Trip_Duration','Start_Time','Start_Station_Latitude','Start_Station_Longitude','Birth_Year','Gender','Holiday','Date']
    
    training_date = ['201706','201707','201708']
    training_database = pd.DataFrame(columns=database_name)
    
    #create training database
    for i in range(len(training_date)):
        
        filename = 'cleaned'+training_date[i]+'.csv'
        print('merging ',filename)
        
        #read in monthly data
        to_add = read_cleaned(filename)
        
        #merge the rides in each month
        training_database = pd.concat([training_database,to_add])
        
    #Add weather data and save.
    add_weather_save(training_database,'duration_prediction_data.csv')
    print('duration data merge complete')   
    
    
   
    
    
    
   
    


merging  cleaned201706.csv
merging  cleaned201707.csv
merging  cleaned201708.csv
duration data merge complete
